In [1]:
import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style()
plt.rcParams['figure.figsize'] = (12, 6)

In [12]:
df = pl.read_csv(
    "../data/processed/df_procesada.csv",
    separator=";",
    columns=["ESTANCIA", "COD_HOSPITAL", "DIAGNOSTICO1", "ANIO_EGRESO"],
)

In [25]:
resumen_egresos_nacionales = (
    df.group_by(pl.col(["ANIO_EGRESO", "COD_HOSPITAL", "DIAGNOSTICO1"]))
    .agg([pl.count().alias("n_egresos"), pl.col("ESTANCIA").sum().alias("dias_estada_totales")])
    .sort(by=["ANIO_EGRESO", "DIAGNOSTICO1", "n_egresos"], descending=True)
    .with_columns(
        pl.col("n_egresos")
        .rank(method="min", descending=True)
        .over(["ANIO_EGRESO", "DIAGNOSTICO1"])
        .alias("ranking_egresos")
    )
)

In [27]:
resumen_egresos_nacionales.filter(
    pl.col("COD_HOSPITAL") == 112103
)

ANIO_EGRESO,COD_HOSPITAL,DIAGNOSTICO1,n_egresos,dias_estada_totales,ranking_egresos
i64,i64,str,u32,i64,u32
2021,112103,"""Z54.0""",1,1,63
2021,112103,"""Z51.2""",2,6,9
2021,112103,"""Z51.1""",3,11,27
2021,112103,"""Z47.0""",5,13,42
2021,112103,"""Z45.0""",109,124,1
2021,112103,"""Z29.8""",4,11,7
2021,112103,"""Z29.2""",4,4,4
2021,112103,"""Z29.1""",2,8,14
2021,112103,"""Z13.7""",1,2,4
